In [1]:
import pandas as pd #đọc dữ liệu
import numpy as np #xử lý dữ liệu
import matplotlib.pyplot as plt #vẽ biểu đồ

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Concatenate, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/PhisingURL/PhiUSIIL_Phishing_URL_Dataset.csv')
print(df)

                                                      URL  URLLength  \
0                        https://www.southbankmosaics.com         31   
1                                https://www.uni-mainz.de         23   
2                          https://www.voicefmradio.co.uk         29   
3                             https://www.sfnmjournal.com         26   
4                      https://www.rewildingargentina.org         33   
...                                                   ...        ...   
235790                     https://www.skincareliving.com         29   
235791                      https://www.winchester.gov.uk         28   
235792                    https://www.nononsensedesign.be         30   
235793  https://patient-cell-40f5.updatedlogmylogin.wo...         55   
235794                 https://www.alternativefinland.com         33   

                                                 Domain  DomainLength  \
0                              www.southbankmosaics.com       

In [ ]:
datas=df["URL"]
label=df["label"]
X_train=datas[:5000]
y_train=label[:5000]
X_test=datas[6000:7000]
y_test=label[6000:7000]
print(X_train)
print(y_test)

0         https://www.southbankmosaics.com
1                 https://www.uni-mainz.de
2           https://www.voicefmradio.co.uk
3              https://www.sfnmjournal.com
4       https://www.rewildingargentina.org
                       ...                
4995        https://www.mosaicmadeeasy.com
4996     https://www.tableskirtingclip.com
4997               http://www.cniitiic.com
4998           https://www.boaeditions.org
4999            https://www.lundfloral.net
Name: URL, Length: 5000, dtype: object
6000    0
6001    1
6002    1
6003    0
6004    0
       ..
6995    1
6996    0
6997    0
6998    1
6999    1
Name: label, Length: 1000, dtype: int64


In [ ]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [ ]:
y_test[0]

array([1., 0.], dtype=float32)

In [ ]:
X_train.shape

(5000,)

In [ ]:
tokenizer=Tokenizer(char_level=True)

In [ ]:
def change_data_char(X_train):
  max_sequence_length=max(len(url) for url in X_train)
  tokenizer.fit_on_texts(X_train)
  encoded_sequences=tokenizer.texts_to_sequences(X_train)
  padded=pad_sequences(encoded_sequences,padding="post",maxlen=200)
  return padded
  # padded la ma tran cac ky tu duoc ma hoa trong ca link

In [ ]:
def change_data_word_char(X_train):
  list_word_in_url=[]
  # max_word_unit=0
  # max_word=0
  for url in X_train:
    tokens=re.findall(r'\w+|\S',url)
    # max_word_unit=max(max_word_unit,max(len(a) for a in tokens))
    # max_word=max(max_word,len(tokens))
    token=' '.join(tokens)
    list_word_in_url.append(token)
  tokenizer_word = Tokenizer(filters='')
  tokenizer_word.fit_on_texts(list_word_in_url)
  sequences_word = tokenizer_word.texts_to_sequences(list_word_in_url)
  padded_1=pad_sequences(sequences_word,padding="post",truncating="post",maxlen=200)

  pad_token_sum=[]
  for url in list_word_in_url:
    token_char_unit=tokenizer.texts_to_sequences(url)
    # pad_token_char_unit=pad_sequences(token_char_unit,padding="post",maxlen=max_word_unit)
    pad_token_char_unit=pad_sequences(token_char_unit,padding="post",maxlen=200)
    pad_token_sum.append(pad_token_char_unit)

  # padded_1 la ma tran ma hoa tu trong cac link
  # pad_token_sum la ma tran ma hoa ky tu theo cac tu trong link
  pad_pad_token_sum=pad_sequences(pad_token_sum,padding="post",maxlen=200)
  # pad_pad_token_sum=pad_sequences(pad_token_sum,padding="post",maxlen=max_word)
  pad_pad_token_sum=pad_pad_token_sum

  URL_cw=np.sum(pad_pad_token_sum,axis=2)
  URL_w_cw=URL_cw+padded_1
  return URL_w_cw

In [ ]:
#CNN for URL_w_cw and padded
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input, concatenate

padded=change_data_char(X_train)
input_shape=padded.shape[1]
# Khởi tạo mô hình cho ma trận padded 472x5000
model_1 = Sequential()
model_1.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(input_shape,1)))
model_1.add(MaxPooling1D(pool_size=2,strides=2))
model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_1.add(MaxPooling1D(pool_size=2,strides=2))
model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_1.add(MaxPooling1D(pool_size=2,strides=2))
model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_1.add(MaxPooling1D(pool_size=2,strides=2))
model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_1.add(MaxPooling1D(pool_size=2,strides=2))
model_1.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_1.add(MaxPooling1D(pool_size=2,strides=2))
model_1.add(Flatten())
out_model_1= Dense(128, activation='relu')(model_1.output)
URL_w_cw=change_data_word_char(X_train)
input_shape_1=URL_w_cw.shape[1]
# Khởi tạo mô hình cho ma trận URL_w_cw
model_2 = Sequential()
model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(input_shape_1,1)))
model_2.add(MaxPooling1D(pool_size=2,strides=2))
model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_2.add(MaxPooling1D(pool_size=2,strides=2))
model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_2.add(MaxPooling1D(pool_size=2,strides=2))
model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_2.add(MaxPooling1D(pool_size=2,strides=2))
model_2.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model_2.add(MaxPooling1D(pool_size=2,strides=2))
model_2.add(Flatten())
out_model_2= Dense(128, activation='relu')(model_2.output)
# Kết hợp đầu ra của hai mô hình trước vào một fully connected layer
merged = concatenate([out_model_1, out_model_2])
merged = Dense(128, activation='relu')(merged)
merged = Dense(64, activation='relu')(merged)
# Lớp output với 2 nơ-ron và hàm kích hoạt softmax
output_layer = Dense(2, activation='softmax')(merged)

# Tạo model từ Input và output
combined_model = Model(inputs=[model_1.input, model_2.input], outputs=output_layer)

# Compile model
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# In cấu trúc của mô hình
combined_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 conv1d_input (InputLayer)   [(None, 200, 1)]             0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 198, 64)              256       ['conv1d_input[0][0]']        
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 99, 64)               0         ['conv1d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv1d_6_input (InputLayer  [(None, 200, 1)]             0         []                        

In [ ]:
# Huấn luyện mô hình
print(y_train.shape)
padded=change_data_char(X_train)
URL_w_cw=change_data_word_char(X_train)
combined_model.fit([padded, URL_w_cw], y_train, epochs=10, batch_size=32, validation_split=0.2)

(5000, 2)
Epoch 1/10
125/125 [==============================] - 8s 29ms/step - loss: 0.3791 - accuracy: 0.8645 - val_loss: 0.2829 - val_accuracy: 0.9380
Epoch 2/10
125/125 [==============================] - 3s 27ms/step - loss: 0.0509 - accuracy: 0.9908 - val_loss: 0.0574 - val_accuracy: 0.9870
Epoch 3/10
125/125 [==============================] - 4s 34ms/step - loss: 0.0404 - accuracy: 0.9927 - val_loss: 0.0629 - val_accuracy: 0.9820
Epoch 4/10
125/125 [==============================] - 3s 24ms/step - loss: 0.0416 - accuracy: 0.9905 - val_loss: 0.0533 - val_accuracy: 0.9870
Epoch 5/10
125/125 [==============================] - 3s 23ms/step - loss: 0.0530 - accuracy: 0.9900 - val_loss: 0.1581 - val_accuracy: 0.9640
Epoch 6/10
125/125 [==============================] - 3s 23ms/step - loss: 0.0379 - accuracy: 0.9935 - val_loss: 0.0443 - val_accuracy: 0.9900
Epoch 7/10
125/125 [==============================] - 4s 31ms/step - loss: 0.0308 - accuracy: 0.9940 - val_loss: 0.0641 - val_accura

In [ ]:
predict_padded=change_data_char(X_test)
predict_URL_w_cw=change_data_word_char(X_test)
predictions = combined_model.predict([predict_padded,predict_URL_w_cw])
print(predictions[0:5])
# print(predict_padded[0])

32/32 [==============================] - 1s 7ms/step
[[9.9999994e-01 3.8934679e-08]
 [1.7963601e-02 9.8203635e-01]
 [8.4664812e-03 9.9153352e-01]
 [9.9999982e-01 6.3293960e-08]
 [9.9999994e-01 0.0000000e+00]]


In [ ]:
y_test[0:5]

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
#Chuyển đổi mảng dự đoán thành các nhãn dơn lẻ
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
predicted_url_single = np.argmax(predictions, axis=1)
actual_url_single = np.argmax(y_test, axis=1)
accuracy = accuracy_score(actual_url_single, predicted_url_single)
precision = precision_score(actual_url_single, predicted_url_single)
recall = recall_score(actual_url_single, predicted_url_single)
f1 = f1_score(actual_url_single, predicted_url_single)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.996
Precision: 0.9930915371329879
Recall: 1.0
F1-score: 0.9965337954939341


In [ ]:
print(predicted_url_single[50:60])
print(actual_url_single[50:60])

[0 1 1 1 0 0 1 0 1 0]
[0 1 1 1 0 0 1 0 1 0]


In [ ]:
import csv
import re
import requests
from requests.exceptions import InvalidURL

def check_url(url):
    # Kiểm tra tính hợp lệ của URL bằng biểu thức chính quy
  if re.match(r'^https?://\S+', url):
      try:
          response = requests.head(url,timeout=3)
          return response.status_code==200
      except Exception as e:
          return False
  else:
      return False

# Đọc danh sách các URL từ tệp CSV
url_listt = pd.read_csv('/content/drive/MyDrive/PhisingURL/malicious_phish_LanLon.csv')
url_list=url_listt["url"][50000:60000]

# Thêm scheme vào mỗi URL
url_list_with_scheme = ['http://' + url if not url.startswith(('http://', 'https://')) else url for url in url_list]

# Kiểm tra sự hoạt động của mỗi URL
count=50000
results = []
for url in url_list_with_scheme:
    if check_url(url):
      results.append([url,url_listt["type"][count]])
      # results[url]=check_url(url)
      print(f"{count}:{url}: {'hoạt động' if results else 'không hoạt động'}")
    count+=1
# results=pd.DataFrame.from_dict(results)
results=pd.DataFrame(results, columns=['url', 'type'])
results.to_csv('/content/drive/MyDrive/PhisingURL/malicious_phish_LanLon_6.csv')
# # In kết quả
# for url, result in results.items():
#     print(f"{url}: {'hoạt động' if result else 'không hoạt động'}")


50021:http://olympiademontreal.com/fr/galeries/: hoạt động
50022:http://beefcakesofwrestling.blogspot.com/2010/08/sylvain-grenier-white-album.html: hoạt động
50025:http://absoluteastronomy.com/topics/John_Cabot: hoạt động
50033:http://allmusic.com/artist/handsome-furs-p892838: hoạt động
50078:http://archive.org/stream/northadamsgenera00nort/northadamsgenera00nort_djvu.txt: hoạt động
50089:http://blog.ericbowersphoto.com/2011/02/the-garment-district-frozen-over/: hoạt động
50097:http://wuzam.com/mp3/bo-burnham-mp3-download?page=3: hoạt động
50103:http://my.tbaytel.net/bmartin/detroit1.htm: hoạt động
50113:http://allmusic.com/artist/digital-underground-p29: hoạt động
50115:http://spplodz.nazwa.pl/archiwum/2010_06a.php: hoạt động
50128:http://dannymorganband.com/: hoạt động
50150:http://mikehernandez.blogspot.com/: hoạt động
50178:http://champcarstats.com/year/2002.htm: hoạt động
50206:http://dragg.in/classifieds/?action=search&squery=9th+standard+hindi+text+book+for+karnataka+state+sylla

In [ ]:
import csv
import re
import requests
from requests.exceptions import InvalidURL

def check_url(url):
    # Kiểm tra tính hợp lệ của URL bằng biểu thức chính quy
      try:
          response = requests.head(url,timeout=3)
          return response.status_code==200
      except Exception as e:
          return False

# Đọc danh sách các URL từ tệp CSV
url_listt = pd.read_csv('/content/drive/MyDrive/PhisingURL/phished-urls_CoMien.csv')
url_list=url_listt["Url"]
url_headers=url_listt.columns.tolist()
# Thêm scheme vào mỗi URL
# url_list_with_scheme = ['http://' + url if not url.startswith(('http://', 'https://')) else url for url in url_list]

# Kiểm tra sự hoạt động của mỗi URL
count=0
results = []
for url in url_list:
    if check_url(url):
      results.append(url_listt.iloc[count].tolist())
      # results[url]=check_url(url)
      print(f"{count}:{url}: {'hoạt động' if results else 'không hoạt động'}")
    count+=1
# results=pd.DataFrame.from_dict(results)
results=pd.DataFrame(results, columns=url_headers)
results.to_csv('/content/drive/MyDrive/PhisingURL/phished-urls_CoMien_1.csv')
# # In kết quả
# for url, result in results.items():
#     print(f"{url}: {'hoạt động' if result else 'không hoạt động'}")


1:https://stepp8-c4298.firebaseapp.com/: hoạt động
8:http://inx.lv/9GkK: hoạt động
11:https://cl7zky.webwave.dev/: hoạt động
22:https://k3yg7p.webwave.dev/: hoạt động
38:http://dappsconnection.cc: hoạt động
43:https://piscinaveronza.com/hesh/dpd/98fe2/: hoạt động
46:https://sitebuilder184234.dynadot.com/: hoạt động
50:https://attmall.boxmode.io/: hoạt động
56:https://wwwenterviacredtc.blogspot.com/?gclid=EAIaIQobChMIrouJ_Y-J_AIVBemRCh0A8wn2EAAYASAAEgJOmvD_BwE: hoạt động
57:https://serviceadhelpmeta.firebaseapp.com/: hoạt động
61:https://atttt.boxmode.io/: hoạt động
64:https://bafybeienects2ejjn3ge4nrlatuye5naj77w2xg6klahspapw2goyaxzt4.ipfs.cf-ipfs.com/?clientID=info@jmjgroup.co.in: hoạt động
69:https://app-br-portal-delicias.blogspot.com/?gclid=EAIaIQobChMIzKHVgO_r-wIVBY7ICh3KuQnwEAAYASAAEgL4xvD_BwE: hoạt động
70:https://edavki-durs.firebaseapp.com/: hoạt động
75:https://xyuh1j.webwave.dev/: hoạt động
86:https://e-bank.pro/: hoạt động
87:https://form.jotform.com/221892180514050: hoạt đ

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc dữ liệu từ file CSV vào DataFrame
df = pd.read_csv('/content/drive/MyDrive/PhisingURL/new_data_urls_KoMien.csv')

# Lọc các mẫu có nhãn là bad
df_label_0 = df[df['status'] == 1]

# Lấy mẫu ngẫu nhiên 5000 mẫu từ nhóm nhãn 0
sample_label_0 = df_label_0.sample(n=2000, random_state=42)

# Lọc các mẫu có nhãn là 1
df_label_1 = df[df['status'] == 0]

# Lấy mẫu ngẫu nhiên 5000 mẫu từ nhóm nhãn 1
sample_label_1 = df_label_1.sample(n=8000, random_state=42)

# Kết hợp hai mẫu lại với nhau
final_sample = pd.concat([sample_label_0, sample_label_1])

# Ghi ra file CSV mới
final_sample.to_csv('/content/drive/MyDrive/PhisingURL/data1_new_data_urls_10k.csv', index=False)
# Chia dữ liệu thành tập train và test
train_data, test_data = train_test_split(final_sample, test_size=0.2, random_state=42)
# Ghi ra file CSV mới
train_data.to_csv('/content/drive/MyDrive/PhisingURL/data1_10k_new_data_urls_train.csv', index=False)
# Ghi ra file CSV mới
test_data.to_csv('/content/drive/MyDrive/PhisingURL/data1_10k_new_data_urls_test.csv', index=False)
